# Import Libraries

In [ ]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split


from keras.models import Model
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D 
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50


import cv2
from keras.preprocessing.image import ImageDataGenerator
pd.set_option('display.max_colwidth', -1)

%matplotlib inline

# Loading Dataset
We'll use here the Pandas to load the dataset into memory

In [ ]:
train_df = pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')
train_df['path'] = '../input/state-farm-distracted-driver-detection/imgs/train/' + train_df['classname']+ '/' +train_df['img']
pred_df = pd.read_csv('../input/state-farm-distracted-driver-detection/sample_submission.csv')
pred_df['path'] = '../input/state-farm-distracted-driver-detection/imgs/test/' + pred_df['img']

In [ ]:
y_count=len(train_df['classname'].unique())

## Resnet50

In [ ]:
resnet50 = ResNet50(include_top = False, input_shape = (224,224,3), weights = 'imagenet')

for layer in resnet50.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(resnet50.output)
predictions = Dense(y_count, activation='softmax')(x)

model_resnet50 = Model(inputs = resnet50.input, outputs = predictions)

## Compile Model

In [ ]:
model_resnet50.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model_resnet50.summary()

## Train and Test Split

In [ ]:
X, y = train_df[['path', 'classname']], train_df['classname']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train Generators

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.10,
    brightness_range=[0.6,1.4],
    channel_shift_range=0.7,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
) 

In [ ]:
train_generator_resnet50 = datagen.flow_from_dataframe(
        X_train,  # This is the source directory for training images
        x_col='path',
        y_col='classname',
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        preprocessing_function=preprocess_input
)

In [ ]:
val_generator_resnet50 = datagen.flow_from_dataframe(
        X_test,  # This is the source directory for training images
        x_col='path',
        y_col='classname',
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=32,
        class_mode="categorical",
        shuffle=True,
        preprocessing_function=preprocess_input
)

## Model Fitting

In [ ]:
history_resnet50 = model_resnet50.fit(
      train_generator_resnet50,
     validation_data=val_generator_resnet50,
      steps_per_epoch=100,
      epochs=60,
      verbose=2)

## Plot Loss

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history_resnet50.history['loss'])
plt.plot(history_resnet50.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.show()

## Plot Accuracy

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history_resnet50.history['accuracy'])
plt.plot(history_resnet50.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

## Model Save

In [ ]:
model_resnet50.save('./custom_model.h5')

# Load Model

In [ ]:
model_resnet50.load_weights('../input/weights/custom_model.h5')

# Explainable AI

In [ ]:
from tf_explain.core.grad_cam import GradCAM
from tf_explain.core.vanilla_gradients import VanillaGradients
from tf_explain.core.occlusion_sensitivity import OcclusionSensitivity

In [ ]:
for i in range(10):
    img_path = '../input/state-farm-distracted-driver-detection/imgs/train/c{0}/'.format(i)
    img_path += os.listdir(img_path)[0]
    img_path
    try:
        os.mkdir("c{0}".format(i))
    except OSError as error: 
        print(error) 
        

    
    img1 = cv2.imread(img_path)
    img1 = cv2.resize(img1, (224, 224),interpolation = cv2.INTER_LINEAR)
    data = ([img1], None)
    
    explainer_GradCAM = GradCAM()
    grid = explainer_GradCAM.explain(data, model_resnet50, class_index=0)  # 281 is the tabby cat index in ImageNet
    explainer_GradCAM.save(grid, ".", "./c{0}/grad_cam.png".format(i))
    
    explainer_VanillaGradients = VanillaGradients()
    grid = explainer_VanillaGradients.explain(data, model_resnet50, class_index=0)  # 281 is the tabby cat index in ImageNet
    explainer_VanillaGradients.save(grid, ".", "./c{0}/VanillaGradients.png".format(i))
    
    explainer_OcclusionSensitivity = OcclusionSensitivity()
    grid = explainer_OcclusionSensitivity.explain(data, model_resnet50, class_index=0, patch_size=16)  # 281 is the tabby cat index in ImageNet
    explainer_OcclusionSensitivity.save(grid, ".", "./c{0}/OcclusionSensitivity.png".format(i))
        

In [ ]:
conf_mat_val_gen = datagen.flow_from_dataframe(
        X_test,  # This is the source directory for training images
        x_col='path',
        y_col='classname',
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=32,
        class_mode="categorical",
        shuffle=False,
        preprocessing_function=preprocess_input
)

In [ ]:
y_pred=model_resnet50.predict_generator(conf_mat_val_gen)

In [ ]:
pred = np.argmax(y_pred, axis=1)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=val_generator_resnet50.classes, predictions=pred).numpy()

In [ ]:
classes = ['c{0}'.format(i) for i in range(10)]
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes, 
                     columns = classes)
con_mat_df

In [ ]:
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
plt.savefig('confussion_matrix.png')